In [17]:
# Tratamiento de datos
import pandas as pd
# Almacenar en caché los resultados de funciones en el disco
import joblib
# Metricas para evaluación del modelo
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
# Creación de modelo de maquinas de soporte vectorial
from sklearn.svm import SVR
# Separar los datos entrenamiento y prueba
from sklearn.model_selection import train_test_split
# Optimización de hiperparametros
from sklearn.model_selection import GridSearchCV
# Uso de validacion cruzada
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# Exportamos los datos del archivo dataSetLimpio4 dado que tiene algunas manipulaciones de las columnas en una clasificacion categorica, eliminamos valores nulos, contaminación 0.5 y quitamos registros atipicos
limpiaDataSetCalidadAire = pd.read_csv('data/stage/dataSetLimpio4.csv')
limpiaDataSetCalidadAire.shape

(19447, 25)

In [19]:
# Revisamos el comportamiento por toma de muestras de los datos y usando variación entre columnas.
# Nota: Se estaba presentando buen comportamiento entre algunos meses del dataset, por ende,
# revisamos algunos registros por meses para identificar diferenciaciones.
# Finalmente, dejamos el dataset completo con la línea siguiente:
limpiaDataSetCalidadAire =limpiaDataSetCalidadAire.sample(n = 19447, random_state=123).iloc[:,:]
limpiaDataSetCalidadAire

,anio,mes,dia,hora,pm25,festivo,temperatura,humedad,presion,p1,...,dia_semana_Inicio Semana,dia_semana_Media semana,estacion_Estacion Aranjuez,estacion_Estacion Belen,estacion_Estacion Caldas,estacion_Estacion Itagui,franja_horaria_Madrugada,franja_horaria_Mañana,franja_horaria_Noche,franja_horaria_Tarde
11216,2024,2,27,1,30,0,19,80,852,0,...,1,0,1,0,0,0,1,0,0,0
8084,2024,7,8,4,9,0,15,86,825,0,...,1,0,0,0,1,0,1,0,0,0
1474,2024,3,14,1,31,0,22,84,0,0,...,0,1,0,0,0,1,1,0,0,0
12353,2024,4,24,11,15,0,27,51,851,0,...,1,0,1,0,0,0,0,1,0,0
3915,2024,8,15,1,25,0,20,84,0,0,...,0,1,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13435,2024,6,12,19,10,0,23,72,851,0,...,1,0,1,0,0,0,0,0,1,0
7763,2024,6,23,6,11,0,15,88,827,0,...,0,0,0,0,1,0,0,1,0,0
15377,2024,9,6,21,27,0,23,67,850,0,...,0,1,1,0,0,0,0,0,1,0
17730,2024,5,22,10,29,0,27,61,852,0,...,1,0,0,1,0,0,0,1,0,0


In [20]:
# Separamos los datos de variables independientes y la variable dependiente
X = limpiaDataSetCalidadAire.drop(columns ='temperatura')
y = limpiaDataSetCalidadAire['temperatura']
# Separamos las columnas de las variables independientes de la dependiente
x_cols = X.columns.tolist()

In [21]:
# División de los datos en datos de entrenamiento del modelo, y datos para el testeo
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    train_size=0.7, # 70 % de nuestros datos en entrenamiento y 30% de los datos para el testeo
    random_state=1234,
    shuffle=True
)

In [22]:
#Escalar Variables númericas
pd.set_option('display.float_format', lambda x: '%.4f' % x)
scaler = joblib.load('data/stage/estandarizacionDataSet4.pkl')

# Se escalan los valores del dataset entrenamiento y prueba
X_train[x_cols] = scaler.fit_transform(X_train[x_cols])
X_test[x_cols] = scaler.transform(X_test[x_cols])
# Probamos diferentes kernel con hiperparametros para revisar sus evaluaciones
SVMR_linear = SVR(kernel='linear', C = 1000)
SVMR_Pol2 = SVR(kernel='poly', C = 100, degree = 2)
SVMR_Pol3 = SVR(kernel='poly', C = 100, degree = 3)
SVMR_rbf = SVR(kernel='rbf', C = 100)  # Disminuimos el parametro de regularizacion C para que no se tenga tanta diferencia entre las metricas de evaluación de datos de entrenamiento y testeo
# SVMR_sig = SVR(kernel='sigmoid', C = 1000)  # Observamos que el kernel sigmoide no refleja resultados esperados y lo excluimos para el analisis
# Entrenamos los modelos, dado que tenemos gran cantidad de datos toma un tiempo para la ejecución
SVMR_linear.fit(X_train, y_train)
SVMR_Pol2.fit(X_train, y_train)
SVMR_Pol3.fit(X_train, y_train)
SVMR_rbf.fit(X_train, y_train)
#SVMR_sig.fit(X_train, y_train)
print('Linear   Training: ', SVMR_linear.score(X_train,y_train), 'Test:',r2_score(y_test,SVMR_linear.predict(X_test)))
print('Poly 2   Training: ', SVMR_Pol2.score(X_train,y_train), 'Test:',r2_score(y_test,SVMR_Pol2.predict(X_test)))
print('Poly 3   Training: ', SVMR_Pol3.score(X_train,y_train), 'Test:',r2_score(y_test,SVMR_Pol3.predict(X_test)))
print('rbf      Training: ', SVMR_rbf.score(X_train,y_train), 'Test:',r2_score(y_test,SVMR_rbf.predict(X_test)))
#print('sigmoide Training: ', SVMR_sig.score(X_train,y_train), 'Test:',r2_score(y_test,SVMR_sig.predict(X_test)))

Linear   Training:  0.9033421959303886 Test: 0.8998319690465588
Poly 2   Training:  0.9426041256574764 Test: 0.93953889961687
Poly 3   Training:  0.9564679392259903 Test: 0.9500667699361912
rbf      Training:  0.9647357147383459 Test: 0.9511830070605752


In [23]:
# luego de la ejecución de los modelos con los hiperparametros predeterminados usamos validación cruzada en 5 pliegues
cross_val_score(SVMR_rbf, X, y, cv=5, scoring='r2')

array([-1.900599  , -1.97341579, -1.89668964, -1.87781262, -1.91358167])

Tunning Afinación del modelo

In [24]:
# Nuevamente se asigna a 'X_Completo' y 'y_Completo' los datos entrada y salida respectivamente
X_completo = X
y_completo = y

# Se escalan los datos numéricos de entrada
X_completo[x_cols] = scaler.fit_transform(X_completo[x_cols])
X_completo

,anio,mes,dia,hora,pm25,festivo,humedad,presion,p1,velocidad_prom,...,dia_semana_Inicio Semana,dia_semana_Media semana,estacion_Estacion Aranjuez,estacion_Estacion Belen,estacion_Estacion Caldas,estacion_Estacion Itagui,franja_horaria_Madrugada,franja_horaria_Mañana,franja_horaria_Noche,franja_horaria_Tarde
11216,0.0000,0.1250,0.8667,0.0435,0.6122,0.0000,0.7791,0.9953,0.0000,0.0000,...,1.0000,0.0000,1.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000
8084,0.0000,0.7500,0.2333,0.1739,0.1837,0.0000,0.8488,0.9638,0.0000,0.0000,...,1.0000,0.0000,0.0000,0.0000,1.0000,0.0000,1.0000,0.0000,0.0000,0.0000
1474,0.0000,0.2500,0.4333,0.0435,0.6327,0.0000,0.8256,0.0000,0.0000,0.1667,...,0.0000,1.0000,0.0000,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,0.0000
12353,0.0000,0.3750,0.7667,0.4783,0.3061,0.0000,0.4419,0.9942,0.0000,0.1667,...,1.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000
3915,0.0000,0.8750,0.4667,0.0435,0.5102,0.0000,0.8256,0.0000,0.0000,0.1667,...,0.0000,1.0000,0.0000,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13435,0.0000,0.6250,0.3667,0.8261,0.2041,0.0000,0.6860,0.9942,0.0000,0.3333,...,1.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000
7763,0.0000,0.6250,0.7333,0.2609,0.2245,0.0000,0.8721,0.9661,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000
15377,0.0000,1.0000,0.1667,0.9130,0.5510,0.0000,0.6279,0.9930,0.0000,0.3333,...,0.0000,1.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000
17730,0.0000,0.5000,0.7000,0.4348,0.5918,0.0000,0.5581,0.9953,0.0000,0.0000,...,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000


In [25]:
#Se entena el modelo con múltiples hiperparámetros buscando el mejor

#Se crea una instancia de una SVR Regression
modelsvr = SVR()


# Usamos 5 pliegues
CV = 5

#Se establecen los hiperparámetros para el Kernel lineal
svm_linear = {'C': [0.1, 1],
              'kernel': ['linear']}

#Se establecen los hiperparámetros para el Kernel polinomico
svm_poly = {'C': [0.1, 1],
              'gamma': [1, 0.1, 'auto', 'scale'],
              'degree': [2, 3],
              'kernel': ['poly']}

#Se establecen los hiperparámetros para los kernels rbf y sigmoid
svm_others = {'C': [0.1, 1],
              'gamma': [1, 0.1,'auto', 'scale'],
              'kernel': [ 'rbf']}

#Se unen todos los hiperparametros en una lista de diccionarios
parameters = [svm_linear, svm_poly, svm_others]

# Se define un GridSearchCV con una validacion cruzada de 10 pliegues y con todos los
#hierparametros establecidos anteriormente
grid_svr = GridSearchCV(modelsvr, param_grid=parameters, cv=CV, verbose=3)

#Se entrena el GridSearchCV anterior (2 min. aprox.) con los datos completos
grid_svr.fit(X_completo,y_completo)

Fitting 5 folds for each of 26 candidates, totalling 130 fits
[CV 1/5] END ..............C=0.1, kernel=linear;, score=0.895 total time=  15.6s
[CV 2/5] END ..............C=0.1, kernel=linear;, score=0.901 total time=  15.5s
[CV 3/5] END ..............C=0.1, kernel=linear;, score=0.902 total time=  15.9s
[CV 4/5] END ..............C=0.1, kernel=linear;, score=0.899 total time=  16.0s
[CV 5/5] END ..............C=0.1, kernel=linear;, score=0.899 total time=  16.3s
[CV 1/5] END ................C=1, kernel=linear;, score=0.896 total time=  19.4s
[CV 2/5] END ................C=1, kernel=linear;, score=0.901 total time=  18.0s
[CV 3/5] END ................C=1, kernel=linear;, score=0.903 total time=  19.0s
[CV 4/5] END ................C=1, kernel=linear;, score=0.901 total time=  20.1s
[CV 5/5] END ................C=1, kernel=linear;, score=0.900 total time=  20.7s
[CV 1/5] END C=0.1, degree=2, gamma=1, kernel=poly;, score=0.932 total time=  19.2s
[CV 2/5] END C=0.1, degree=2, gamma=1, kerne

GridSearchCV(cv=5, estimator=SVR(),
             param_grid=[{'C': [0.1, 1], 'kernel': ['linear']},
                         {'C': [0.1, 1], 'degree': [2, 3],
                          'gamma': [1, 0.1, 'auto', 'scale'],
                          'kernel': ['poly']},
                         {'C': [0.1, 1], 'gamma': [1, 0.1, 'auto', 'scale'],
                          'kernel': ['rbf']}],
             verbose=3)

In [26]:
# Obtener los resultados del grid search en un dataframe para manipular
results_grid_svr = pd.DataFrame(grid_svr.cv_results_)


# Seleccionar las columnas deseadas
columns_grid_svr = ['param_C']  + \
                    ['param_degree']  + \
                    ['param_kernel']  + \
                    ['param_gamma']   + \
                    ['mean_test_score', 'std_test_score']  + \
                    [f'split{i}_test_score' for i in range(CV)]

# Filtrar y mostrar los resultados
results_grid_svr_filtered = results_grid_svr[columns_grid_svr]

# Crear la columna scoreWithStd: f1 / std
results_grid_svr_filtered['scoreWithStd'] = results_grid_svr_filtered.apply(
    lambda row: row['mean_test_score'] / row['std_test_score'] if row['std_test_score'] != 0 else 0,
    axis=1
)

# Encuentra el índice del máximo valor en la columna scoreWithStd
indice_max_scoreWithStd = results_grid_svr_filtered['scoreWithStd'].idxmax()

# Mostrar los scores promedios por cada parámetro
print(results_grid_svr_filtered[['param_C', 'param_degree', 'mean_test_score', 'std_test_score', 'scoreWithStd']])

    param_C  param_degree  mean_test_score  std_test_score  scoreWithStd
0    0.1000           NaN           0.8991          0.0024      372.8583
1    1.0000           NaN           0.9002          0.0024      372.0314
2    0.1000        2.0000           0.9362          0.0021      440.2392
3    0.1000        2.0000           0.8379          0.0051      163.3298
4    0.1000        2.0000           0.6356          0.0068       93.3523
5    0.1000        2.0000           0.9136          0.0019      472.3373
6    0.1000        3.0000           0.9464          0.0015      652.0698
7    0.1000        3.0000           0.7733          0.0068      113.0115
8    0.1000        3.0000           0.2650          0.0059       45.2144
9    0.1000        3.0000           0.9239          0.0019      475.2114
10   1.0000        2.0000           0.9387          0.0020      458.5822
11   1.0000        2.0000           0.9188          0.0020      449.2550
12   1.0000        2.0000           0.8723         

<ipython-input-26-4471c8a65a28>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_grid_svr_filtered['scoreWithStd'] = results_grid_svr_filtered.apply(


In [27]:
# Se identifican los mejores hiperparámetros
best_params = grid_svr.best_params_
print('Best Parameters : ',best_params)

Best Parameters :  {'C': 1, 'degree': 3, 'gamma': 1, 'kernel': 'poly'}


In [28]:
# Top 3 de los mejores scores
results_grid_svr_filtered.sort_values(by='scoreWithStd', ascending=False).head(3)[
    ['param_C', 'param_degree', 'param_kernel','param_gamma','mean_test_score', 'std_test_score', 'scoreWithStd']]

,param_C,param_degree,param_kernel,param_gamma,mean_test_score,std_test_score,scoreWithStd
14,1.0000,3.0000,poly,1,0.9507,0.0011,848.3652
6,0.1000,3.0000,poly,1,0.9464,0.0015,652.0698
17,1.0000,3.0000,poly,scale,0.9422,0.0018,520.8159


In [29]:
# Seleccionar el registro correspondiente al índice máximo
registro_max_scoreWithStd = results_grid_svr_filtered.loc[indice_max_scoreWithStd]

registro_max_scoreWithStd.transpose()

,14
param_C,1.0000
param_degree,3.0000
param_kernel,poly
param_gamma,1
mean_test_score,0.9507
std_test_score,0.0011
split0_test_score,0.9489
split1_test_score,0.9501
split2_test_score,0.9508
split3_test_score,0.9519


In [30]:
# Se crea un modelo SVR con los mejores hiperparámetros
modelsvr = SVR( kernel = best_params["kernel"]
                    , gamma = best_params["gamma"]
                    , C= best_params["C"]
                    , degree = best_params["degree"])


#Se entrena el modelo con los datos completos
modelsvr.fit(X_completo, y_completo)

SVR(C=1, gamma=1, kernel='poly')

In [31]:
#Se guardan los modelos de SVC

joblib.dump(modelsvr, 'data/analytics/modelosTemperatura/SVR_CV_varios_cal_aire_temp.pkl')

['drive/MyDrive/Esp_data_science/ML1/modelos/regresion/SVR_CV_varios_cal_aire_temp.pkl']